# Đồ án môn học Lập trình song song ứng dụng

# Phát hiện khuôn mặt bằng Cascaded-Classifiers

## Giáo viên: Trần Trung Kiên

## Sinh viên:
- 1712628 - Huỳnh Hiếu Nhân
- 1712657 - Chung Kiệt Phong
- 1712869 - Đào Duy Tuấn

---

# 1. Mô tả ứng dụng

- Nhóm em chọn ứng dụng Phát hiện khuôn mặt bằng Cascaded-Classifiers
	- Input: 1 ảnh gồm nhiều khuôn mặt
	- Output: cũng là bức ảnh đó nhưng được gắn thêm các "window" không trùng nhau cho tất cả khuôn mặt (không chồng chèo lên nhau)
	- Ý nghĩa của ứng dụng trong thực tế:
  + Ứng dụng để tự động đề xuất gắn thẻ cho ảnh trên mạng xã hội.
  + Ứng dụng để cải thiện tốc độ AF-Tracking (Auto Focus) trên máy ảnh, điện thoại thông minh.
  + Nâng cao hơn là nhận diện thời gian thực thông qua camera giám sát (nằm ngoài đồ án môn học này).
- Lý do cần tăng tốc ứng dụng: ứng dụng sẽ chạy chậm nếu cài đặt tuần tự. Vì khi số lượng khuôn mặt quá nhiều thì sẽ phải đợi rất lâu mới có kết quả, trong khi trong thực tế người ta cần độ chính xác cao và tốc độ nhanh để nhận diện ngay lập tức.
- Tiềm năng song song hóa của ứng dụng: từng vị trí khi bounding-box dịch chuyển là độc lập nhau, nếu chạy tuần tự sẽ mất nhiều thời gian => có tiềm năng song song hoá.
(Nhận định sơ bộ thôi; ví dụ, ứng dụng huấn luyện Convolutional Neural Network có tiềm năng song song hóa vì trong đó có phép convolution, phép nhân ma trận, ...)


---